# Pre-set networks

To speed up experimentation, but above all to ensure better reproducibility of experiments, pre-designed networks have been implemented in sumo-experiments. There are currently three types of pre-designed network :
- A network composed with only one intersection (OneCrossroadNetwork)
- A network composed of $x$ intersections arranged in a line (LineNetwork)
- A network composed of $x \times y$ crossroads arranged in a square (SquareNetwork)

Each network is represented by a class in the `preset_networks` package. The classes contains methods that generates infrastructures, flows, and all add-ons that can be added to a network.

***Note :** For now, only detectors can be added to a pre-set network. Adding new features is one of our priority.*

Let's see how we create a simulation with a pre-set network. We will use the `OneCrossroadNetwork` to do this.

First, we must instanciate the preset network.

In [1]:
from sumo_experiments.preset_networks import OneCrossroadNetwork

network = OneCrossroadNetwork()

## Methods

### Infrastructures

Each class contains one method that generates infrastructures (`generate_infrastructures`). It is the function that you must use to create an Experiment. This function return an `InfrastructureBuilder` object, containing all the infrastructures needed for the definition of the network (nodes, edges, connections, traffic light programs). Thus, in this example with the `OneCrossroadNetwork`, the function will return an object containing 5 nodes (including 1 traffic light), 8 edges (4 * 2 directions), and 12 connections.

The method `generate_infrastructures` has multiple parameters that allows to custom the network. The `lane_length` is the length of all edges of the network, in meters. The `green_time` and the `yellow_time` are respectively the duration of the green and yellow phases, in seconds. The `max_speed` is the maximum speed allowed on the network.

***Note :** The SquareNetwork class also implements a `generate_random_infrastructures` method. It also implements a square network, but with random road length (between the config parameters `minimum_edge_length` and `maximum_edge_length`).*

In [2]:
infrastructures = network.generate_infrastructures(
    lane_length = 200,
    green_time = 60,
    yellow_time = 3,
    max_speed = 50
)

### Flows

Each class contains three methods to define flows for the network :
- The `generate_flows_only_ahead` method generates flows of vehicles that enter from an entry and can only leave the network by the opposite exit. This makes the vehicles unable to turn at intersections. The parameters `stop_generation_time` and `flow_frequency` set the simulation step when flows will end and the vehicle generation frequency, in vehicle/hour/entry (and not routes). Vehicles are generated following a distribution law, which can be 'binomial' (default) or 'uniform'.
- The `generate_flows_all_directions` method generates flows for each entry, and the vehicles can leave the network by any exit (except the one that is related to the entry). Vehicles can then turn at intersections. The parameters are the exact same as `generate_flows_only_ahead`.
- The `generate_flows_with_matrix` method generate variable flows over time that can go anywhere in the intersections. We'll see this in a future section.

Each method returns a `FlowBuilder` object containing all the flows and informations about it (flow entry, exit, frequency, vehicle type).

Let's do some graphical experiments to show you the difference between the `generate_flows_only_ahead` and the `generate_flows_all_directions`.

In [4]:
flows_only_ahead = network.generate_flows_only_ahead(
    stop_generation_time=600,
    flow_frequency=300,
    distribution='binomial'
)

flows_all_direction = network.generate_flows_all_directions(
    stop_generation_time=600,
    flow_frequency=300,
    distribution='binomial'
)

In [6]:
from sumo_experiments import Experiment

exp_only_ahead = Experiment(
    name = 'only_ahead',
    infrastructures = infrastructures,
    flows = flows_only_ahead
)

exp_all_directions = Experiment(
    name = 'all_directions',
    infrastructures = infrastructures,
    flows = flows_all_direction
)

exp_only_ahead.run(
    simulation_duration=600, 
    gui=True
)

exp_all_directions.run(
    simulation_duration=600, 
    gui=True
)

exp_only_ahead.clean_files()
exp_all_directions.clean_files()

Success.
Success.
Success.
Success.


### Detectors

Optionally, you can add detectors to the intersections of a network to adapt their operation to the traffic. While detectors can be added when generating the pre-set network, adapting the behavior of traffic lights is more complex and will be the subject of another tutorial. Detectors' range appears in blue in SUMO GUI.

Each pre-set network has three methods to generate detectors :
- `generate_boolean_detectors` generates short-range detectors that check if a car is its range or not. You can set the range of the detector with the parameter `boolean_detector_length`.
- `generate_numerical_detectors` generates long-range detectors (all of the edge) that are supposed to count vehicles that are on its range. The range of numerical detectors is all the length of the edge it observes.
- `generate_all_detectors` generates the boolean and the numerical detectors for each traffic light.

The detectors generates data that can be used by the TraCi library to modify the traffic lights behaviour.

To add detectors to a network, just add the `detectors` parameter to an experiment, with the right function to use to generate it.

In [7]:
boolean_detectors = network.generate_boolean_detectors(
    boolean_detector_length=20
)

In [9]:
exp_detectors = Experiment(
    name = 'detectors',
    infrastructures = infrastructures,
    flows = flows_only_ahead,
    detectors = boolean_detectors
)

exp_detectors.run(
    simulation_duration=600,
    gui=True
)

exp_detectors.clean_files()

Success.
Success.


***Note :** Some other features should be added in the future, like pedestrians, bicycles, etc.*

**In the next tutorial, we'll see how to create experiments with variable flows, using the `generate_flows_with_matrix` from a pre-set network object.**